In [47]:
import pandas as pd
import numpy as np

In [11]:
from speedml import Speedml
sml = Speedml('currentTestingData.csv',
              'forAutoEda_target.csv',
              target='Engagements')

sml.eda()

,Results,Observations
Speedml Release,v0.9.3,Visit https://speedml.com for release notes.
Nulls,NaN,Use feature.impute.
Outliers Upper,"[bottom3Tagged, Morning, 6, 7, 8, 9]",Positive skew (> 3). Use feature.outliers(upper).
Shape,"train (7766, 36) | test (7765, 1)",
Numerical High-cardinality,"[NBA_Follower_Count, Len_Desc, propOfSpecialTe...",(>10) categories. Use feature.density
Numerical Categorical,"[Tagged_Count, teamTagged, otherTagged, top3Ta...",Use plot.ordinal.
Numerical Continuous,[Engagements],~80% unique. Use plot.continuous.
Target Analysis (Engagements),Model ready.,Use regression models.


In [36]:
data = pd.read_csv('training_set.csv', encoding = "ISO-8859-1")

In [37]:
data.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [38]:
data['Description']= data.Description.fillna('')

In [39]:
data['len_desc']= data['Description'].str.len()

In [40]:
data = data.join(pd.get_dummies(data['Type']))

In [41]:
features =['Video','len_desc']
target = ['Engagements']

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2)

In [43]:
# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    

In [60]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import fbeta_score, make_scorer
def evaluate_model(estimator, features, target, data):
    return cross_validate(estimator,                     
                    X=data[features],
                    y=data[target],
                    scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False),
                    n_jobs=-1, cv=3,
                    return_train_score=True)

def display_results(results):
    results_df  = pd.DataFrame(results).T
    results_cols = results_df.columns
    for col in results_df:
        results_df[col] = results_df[col].apply(np.mean)
    return results_df



In [45]:
RESULTS = {}

In [48]:
from sklearn.ensemble import GradientBoostingRegressor
RESULTS["xgb"] = evaluate_model(GradientBoostingRegressor())


In [49]:
display_results(RESULTS)

,fit_time,score_time,test_score,train_score
xgb,0.177268,0.421699,-38.560176,-38.639255


In [53]:
test2 = pd.read_csv('furtherValidationsNoStandardize.csv')

In [54]:
test2.head()

,NBA_Follower_Count,Len_Desc,Tagged_Count,propOfSpecialTexts,postsInThatDay,teamTagged,otherTagged,top3Tagged,bottom3Tagged,hasHashtag,...,4,5,6,7,8,9,10,11,12,Engagements
0,36984682,95,4,0.266667,5,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,502093
1,36984682,64,2,0.166667,5,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,603380
2,36984682,46,2,0.250000,5,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,603380
3,36984682,43,0,0.000000,5,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,725100
4,36984682,57,1,0.100000,5,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,661446


In [55]:
test2.columns

Index(['NBA_Follower_Count', 'Len_Desc', 'Tagged_Count', 'propOfSpecialTexts',
       'postsInThatDay', 'teamTagged', 'otherTagged', 'top3Tagged',
       'bottom3Tagged', 'hasHashtag', 'Video', 'Not_Video', 'Afternoon',
       'Evening', 'Morning', 'Night', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', 'Engagements'],
      dtype='object')

In [66]:
features2 = ['NBA_Follower_Count', 'Len_Desc', 'Tagged_Count', 'propOfSpecialTexts',
       'postsInThatDay', 'teamTagged', 'otherTagged', 'top3Tagged',
       'bottom3Tagged', 'hasHashtag', 'Video', 'Not_Video', 'Afternoon',
       'Evening', 'Morning', 'Night', 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Thursday', 'Tuesday', 'Wednesday', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12']
target2 =['Engagements']

In [67]:
RESULTS["xgb"] = evaluate_model(GradientBoostingRegressor(),features2,target2,test2)

In [68]:
display_results(RESULTS)

,fit_time,score_time,test_score,train_score
xgb,0.585633,0.216075,-39.812585,-40.202924
